# Road Following Car
First import the library

In [ ]:
from jetbot import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as ipywidgets
from jetbot import Camera
import uuid
import subprocess
from jetbot import Robot
from IPython.display import display
import sys
import math
import cv2 as cv
import numpy as np
import io
from PIL import Image
from uuid import uuid1 #to generate file names
import os
import time
from PIL import Image as im

Create the robot instance as well as the Camera instance.
Make sure that the robot is not moving in order to avoid accidents once the code starts running

In [ ]:
robot = Robot()
camera = Camera.instance(width=300, height=300) #initializes the camera instance
robot.stop()

Define functions in order to process images. <br>
`convert_image_to_numpy` is a function made to transform the images into numpy array for preprocessing. <br>
`save_snapshot` saves images into a pre-set directory.

In [3]:
def convert_image_to_numpy(image):
    imageBinaryBytes = image.value
    imageStream = io.BytesIO(imageBinaryBytes)
    imageFile = Image.open(imageStream)
    I = np.asarray(imageFile)
    return I

from uuid import uuid1 #to generate file names

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg') 
    
    with open(image_path, 'wb') as f:
        f.write(image.value)

Use the python notebook widgets library in order to be able to stream what the camera is capturing.

In [4]:
image = ipywidgets.Image(format='jpeg', width=300, height=300)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Looping Code:
1. Identify the black and green pixels in the image (rgb was determined through trial and error and using paint)
2. Pre process the image by removing the noise
    - use erode to remove the small white noises
        - https://tinyurl.com/s45zmj5a
    - use dilate after doing erosion to return the image to the original size. It helps to join the the broken parts of an object
        - https://appdividend.com/2022/03/15/python-cv2-dilate/
    - if no pixels fall under our black rgb set range for a predetermined number of frames we stop and finish the process
3. For the sake of debugging and understanding our measurements we draw contours onto the image being displayed on the notebook
4. Calculate the angle and center of the boxes
5. Send the "error" distance from center and angle to the PD controller

In [ ]:
integral_ang=0
integral_err=0
ang=0
image = ipywidgets.Image(format='jpeg', width=300, height=300)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
### Not sure we need this
count=0
while(True):

    # 1 isolate blackline
    img_arr = convert_image_to_numpy(image)
    roi = img_arr[200:300, 0:300]
    blackline= cv.inRange(roi,(0,0,0),(68,68,68)) # use rgb values to identify the black lines
    greenbox = cv.inRange(roi, (0,75,0), (90,150,90)) # use rgb values to identify the green squares

    # 2 remove noise by convoluting
    kernel = np.ones((3,3), np.uint8)
    # moves a kernel through the image as in 2D convolution to remove white noises
    blackline = cv.erode(blackline, kernel, iterations=5) 
    blackline = cv.dilate(blackline, kernel, iterations=9)
    greenbox = cv.erode(greenbox, kernel, iterations=5)
    greenbox = cv.dilate(greenbox, kernel, iterations=9)

    # 3 find contours
    contours_blk, hierarchy_blk = cv.findContours(blackline.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours_grn, hierarchy_grn = cv.findContours(greenbox.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    # draw contours, bounding box, and center line
    greend= False
    # centb=0
    x_last = 300
    y_last = 150

    # no black was detected
    if len(contours_blk) == 0 : 
        count=count+1
        # if the black line is not detected for count number of frames then we stop moving and finish the program
        if count == 5:
            newimage = im.fromarray(roi)
            display(newimage)
            print("error")
            robot.stop()
            break

    # black is detected
    if len(contours_blk) > 0 :

        # only one black shape was detected
        if len(contours_blk)==1:
            blackbox=cv.minAreaRect(contours_blk[0])

        else:
            cands=[] # list of candidates
            off_bottom=0

            # Analyze every black object detected
            for num in range(len(contours_blk)):
                # determine the best fitted box around the object
                blackbox = cv.minAreaRect(contours_blk[num])
                (x_min, y_min), (w_min, h_min), ang = blackbox
                box = cv.boxPoints(blackbox)
                (x_box,y_box) = box[0] # gets the lowest corner of the box

                # check if it falls in roi (roi = bottom of image)
                if y_box > 298 :
                    off_bottom += 1

                # append the values of the box, the idx of the contour, and the x_min, y_min to candidates    
                cands.append((y_box,num,x_min,y_min))
            

            # sort the candidates according to the y_box bottom to top
            cands=sorted(cands)
            if off_bottom>1:
                cands_ob=[]
                for num in range((len(contours_blk)-off_bottom),len(contours_blk)):
                    (y_highest,con_highest,x_min, y_min) = cands[num]
                    total_distance = (abs(x_min - x_last)**2 + abs(y_min - y_last)**2)**0.5
                    cands_ob.append((total_distance,con_highest))
                    cands_ob = sorted(cands_ob)         
                    (total_distance,con_highest) = cands_ob[0]         
                    blackbox = cv.minAreaRect(contours_blk[con_highest])
            else:
                (y_highest,con_highest,x_min, y_min) = cands[len(contours_blk)-1]
                blackbox = cv.minAreaRect(contours_blk[con_highest]) 
        (x_min, y_min), (w_min, h_min), ang = blackbox        
        x_blk,y_blk,w_blk,h_blk = cv.boundingRect(contours_blk[0])
        centb=x_min+(w_min/2)
        x_last = x_min 
        y_last = y_min
        if ang < -45 :
            ang = 90 + ang
        if w_min < h_min and ang > 0:
            ang = (90-ang)*-1
        if w_min > h_min and ang < 0:
            ang = 90 + ang

    if len(contours_grn) > 0 :
        x,y,w,h = cv.boundingRect(contours_grn[0])
        centg=x+(w/2)
        cv.rectangle(roi,(x,y),(x+w,y+h),(0,0,255),3)
        cv.line(roi, (int(x+(w/2)), 200), (int(x+(w/2)), 250),(255,0,0),3)
        if y_last> y:
            greend= True

    if greend:
        if centg-centb > 5 :
            cv.putText(roi, "Turn Right", (50,50), cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3)
            robot.right(0.3)
            time.sleep(0.1)
            robot.stop()
            #robot.forward()
        else :
            if centg-centb > 0 :
                cv.putText(roi, "U-Turn", (50,50), cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3)
                robot.right(0.3)
                time.sleep(1)
                robot.stop()
                #robot.forward()
            else:
                cv.putText(roi, "Turn left", (50,50), cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3)
                robot.left(0.3)
                time.sleep(0.1)
                robot.stop()
                robot.forward()
        box = cv.boxPoints(blackbox)
        box = np.int0(box)
        cv.drawContours(roi,[box],0,(0,0,255),3)
        cv.putText(roi,str(ang),(10, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv.putText(roi,str(error),(50, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv.line(roi, (int(x_min),0 ), (int(x_min),100 ), (255,0,0),3)

    else:
        setpoint = 150
        error = int(x_last - setpoint) 
        ang = int(ang) 
        box = cv.boxPoints(blackbox)
        box = np.int0(box)
        cv.drawContours(roi,[box],0,(0,0,255),3)
        cv.putText(roi,str(ang),(10, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv.putText(roi,str(error),(50, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv.line(roi, (int(x_min),0 ), (int(x_min),100 ), (255,0,0),3)



        speed=0.24
        kp=0.1
        ap=0.3
        steer= error*kp + ang * ap
        if error>100:
            steer=steer*0.7
        # if line is to the left of jetbot, turn left
        if steer>0:
            steer=10-steer
            robot.set_motors(speed,speed*steer/10)
        # if line is to the right of jetbot, turn right
        else: 
            steer=steer*-1
            steer=10-steer
            robot.set_motors(speed*steer/10,speed)

    newimage = im.fromarray(roi)
    display(newimage)

In [9]:

#     x_slider.value = x_min
#     speed_slider.value = speed_gain_slider.value
#     pid = ang * steering_gain_slider.value + (ang-angle_last) * steering_dgain_slider.value
#     #pid = x_min * steering_gain_slider.value + (x_min - setpoint) * steering_dgain_slider.value + ang * steering_gain_slider.value + (ang - angle_last) * steering_dgain_slider.value
#     steering_slider.value = pid + steering_bias_slider.value
#     robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
#     robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
#     angle_last= ang

#     integral_ang+=ang
#     integral_err+=error
    
#     elif ang-angle_last<-50:
#         robot.left(0.7)
#     elif error-error_last<-50:
#         robot.left(0.7)
#     elif error-error_last>50:
#         robot.right(0.7)
#     elif ang < -2:
#         robot.left(0.1)
        
#     # if line is to the right of jetbot, turn right
#     elif ang > 2:
#         robot.right(0.1)
        
#     # if line is straight ahead of jetbot, move forward
#     else:
#         if error>20:
#             robot.right(0.1)
#         elif error<-20:
#             robot.left(0.1)
#         else:
#             robot.forward(0.4)

#     kp=0.75
#     ap=1
#     right_speed=0.35
#     left_speed=right_speed+0.02
#     steering= error*kp + ang * ap
#     if steering >-0.5 and steering<0.5:
#         robot.set_motors(left_speed,right_speed)
#     else:
#         if steering>0:
#             steering=100-steering
#             robot.set_motors(left_speed,right_speed*steering/100)
#         else:
#             steering=steering*-1
#             steering= 100-steering
#             robot.set_motors(left_speed*steering/100,right_speed)

In [ ]:
# if steer>1:
#         robot.set_motors(speed*steer,speed)
#         cv.putText(roi,"l1",(50, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     elif steer>0:
#         robot.set_motors(speed*(1/steer),speed)
#         cv.putText(roi,"l2",(50, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     elif steer == 0:
#         robot.forward(speed)
#         cv.putText(roi,"f",(50, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     elif steer <-1: 
#         steer=steer*-1
#         robot.set_motors(speed,speed*steer)
#         cv.putText(roi,"r1",(50, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     elif steer<0:
#         steer=steer*-1
#         robot.set_motors(speed,speed*(1/steer))
#         cv.putText(roi,"r2",(50, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     cv.putText(roi,str(steer),(200, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

We do not need this ###################################

In [ ]:
robot.left(0.1)

In [ ]:

from PIL import Image as im
roi = im.fromarray(roi)
display(roi)

In [ ]:
robot.stop()

In [ ]:
#isolate blackline
image = convert_image_to_numpy(image)
#roi = image[200:250, 0:300]
blackline= cv.inRange(image,(0,0,0),(50,50,50))
greenbox = cv.inRange(image, (0,65,0), (100,200,100))

In [ ]:
#remove noise by convoluting
kernel = np.ones((3,3), np.uint8)
blackline = cv.erode(blackline, kernel, iterations=5)
blackline = cv.dilate(blackline, kernel, iterations=9)
greenbox = cv.erode(greenbox, kernel, iterations=5)
greenbox = cv.dilate(greenbox, kernel, iterations=9)

In [ ]:
#find contours
contours_blk, hierarchy_blk = cv.findContours(blackline.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
contours_grn, hierarchy_grn = cv.findContours(greenbox.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

In [9]:
#draw contours, bounding box, and center line
greend= False
cv.drawContours(image, contours_blk, -1, (0,255,0), 3)
#centb=0
if len(contours_blk) > 0 :
    x,y,w,h = cv.boundingRect(contours_blk[0])
    centb=x+(w/2)
    cv.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)
    cv.line(image, (int(x+(w/2)), 0), (int(x+(w/2)), 250),(255,0,0),3)

if len(contours_grn) > 0 :
    greend= True
    x,y,w,h = cv.boundingRect(contours_grn[0])
    centg=x+(w/2)
    cv.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)
    cv.line(image, (int(x+(w/2)), 0), (int(x+(w/2)), 250),(255,0,0),3)

if greend:
    if centg-centb > 10 :
        cv.putText(image, "Turn Right", (250,180), cv.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0),3)
    else :
        if centg-centb > 0 :
            cv.putText(image, "U-Turn", (50,180), cv.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0),3)
        else:
            cv.putText(image, "U-Turn", (50,180), cv.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0),3)
else:
    setpoint = 150
    error = centb - setpoint
    centertext = "Error = " + str(error)
    cv.putText(image, centertext, (200,340), cv.FONT_HERSHEY_SIMPLEX, 1.5, (255,0,0),3)

TypeError: Expected Ptr<cv::UMat> for argument '%s'

In [ ]:
from PIL import Image as im
newimage = im.fromarray(image)
display(newimage)

In [ ]:
roi=im.fromarray(roi)
display(roi)

In [ ]:
#################################################################################################